In [ ]:
%load_ext autoreload
%autoreload 2


import json
import os

from flask import Flask
from flask import request

import psycopg2
import boto3
from research.weight_estimation.population_metrics import PopulationMetricsEstimator
from research.lice_counting.lice_population_metrics import LicePopulationMetricsEstimator

In [ ]:
def get_db_params_from_aws(data_store):

    # ssm = boto3.client('ssm', region_name='eu-west-1')
    # host_param = ssm.get_parameter(Name='/DW_DB_RO_HOST', WithDecryption=True)
    # host = host_param['Parameter']['Value']
    #
    # user_param = ssm.get_parameter(Name='/DW_DB_RO_USER', WithDecryption=True)
    # user = user_param['Parameter']['Value']
    #
    # password_param = ssm.get_parameter(Name='/DW_DB_RO_PASSWORD', WithDecryption=True)
    # password = password_param['Parameter']['Value']
    #
    # dbname_param = ssm.get_parameter(Name='/DW_DB_RO_NAME', WithDecryption=True)
    # dbname = dbname_param['Parameter']['Value']

    import os
    if data_store == 'DATA_WAREHOUSE':
        credentials = json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS']))
    elif data_store == 'PROD_DB':
        credentials = json.load(open(os.environ['PROD_SQL_CREDENTIALS']))
    else:
        raise Exception('Invalid data store passed')

    host, user, password, dbname = credentials['host'], credentials['user'], credentials['password'],\
                                   credentials['database']

    print("Successfully retrieved DB params.")
    return host, user, password, dbname

def get_lice_data_from_db(pen_id, dates_to_include):

    # get sampling schedule by pen ID (default to 24 hour sampling if pen not available)
    date_list = '({})'.format(", ".join([f"'{d}'" for d in dates_to_include]))
    print("Connecting to DB...")
    host, user, password, dbname = get_db_params_from_aws('PROD_DB')
    res = []
    conn = None
    try:
        conn = psycopg2.connect("dbname="+dbname+" user="+user+" host="+host+" password="+password)
        cur = conn.cursor()

        query = """
            SELECT 
            to_char(captured_at, 'YYYY-MM-DD') AS date,
            annotation_metadata
            FROM annotations
            WHERE pen_id={0}
            AND to_char(captured_at, 'YYYY-MM-DD') IN {1}
            AND is_qa=TRUE
            AND is_skipped=FALSE;
        """.format(pen_id, date_list)

        # execute statement
        cur.execute(query)

        # fetch rows
        rows = cur.fetchall()
        for row in rows:
            res.append(row)

        cur.close()
    except psycopg2.DatabaseError as error:
        print(error)
        print("COULD NOT CONNECT TO DB")
    finally:
        if conn is not None:
            conn.close()

    print(f"Successfully retrieved from DB: {len(res)} rows")
    return res


def get_per_fish_lice_counts(lice_anns):

    per_fish_lice_counts = []
    for date, ann in lice_anns:
        if ann:
            adult_female_count = ann['liceCounts']['adultFemaleCountAdjusted']
            mobile_count = ann['liceCounts']['movingCountAdjusted']
            scottish_adult_female_count = ann['liceCounts']['scottishAdultFemaleCountAdjusted']
        else:
            adult_female_count, mobile_count, scottish_adult_female_count = 0, 0, 0
        per_fish_lice_counts.append(dict(
            date=date,
            adult_female_count=adult_female_count,
            mobile_count=mobile_count,
            scottish_adult_female_count=scottish_adult_female_count
        ))

    return per_fish_lice_counts


def generate_smart_lice_metrics(data):
    pen_id = data['penId']
    dates_to_compute = sorted(list(data['datesToCompute']))
    dates_to_include = sorted(list(data['datesToInclude']))

    resp = dict()
    lice_anns = get_lice_data_from_db(pen_id, dates_to_include)
    if not lice_anns:
        return resp
    per_fish_lice_counts = get_per_fish_lice_counts(lice_anns)
    lice_pme = LicePopulationMetricsEstimator(per_fish_lice_counts)

    for date in dates_to_compute:
        smart_metrics = lice_pme.generate_smart_metrics_on_date(date, max_day_difference=2)
        resp_for_date = dict(
            stationaryCountSmartAvg=None,
            adultFemaleCountSmartAvg=smart_metrics['adult_female_count_smart_avg'],
            mobileCountSmartAvg=smart_metrics['mobile_count_smart_avg'],
            scottishAdultFemaleCountSmartAvg=smart_metrics['scottish_adult_female_count_smart_avg'],
            numSmartAvgLatiFish=smart_metrics['smart_sample_size']
        )
        resp[date] = resp_for_date
    return resp

In [ ]:
data = {
    "penId": 56,
    "datesToCompute": [
        "2020-05-16",
        "2020-05-17",
        "2020-05-18"
    ],
    "datesToInclude": [
        "2020-05-11",
        "2020-05-12",
        "2020-05-13",
        "2020-05-14",
        "2020-05-15",
        "2020-05-16",
        "2020-05-17",
        "2020-05-18"
    ]
}

In [ ]:
resp = generate_smart_lice_metrics(data)

In [ ]:
resp

In [ ]:
(2.14285714285714 * 7 + 2*1 + 2.66666666666667 * 3 + 1.5*2) / 13